In [7]:
import pandas as pd
from pathlib import Path

crespath = Path("./cres_v2.1")
audiopath = crespath / "audio_files"
audiopath_16000 = crespath / "audio_files_16000"

In [2]:
dfs = []

for split in "train dev test".split():
    path = crespath / f"metadata_labelled_{split}.csv"
    df = pd.read_csv(str(path))
    df["split"] = split
    dfs.append(df)

df = pd.concat(dfs).reset_index(drop=True)

In [28]:
df.head(1)

,utterance_id,gender,age,speaker_id,expression,manual_transcription,discrete_emotion_collection_phase,discrete_emotion_annotation_phase,valence_annotation_phase,arousal_annotation_phase,split
0,165,f,adult,35,real,KAD SI REKO SAMO SMIRI SE,anger,anger,2.728209,8.093269,train


In [29]:
df.discrete_emotion_annotation_phase.unique()

array(['anger', 'neutral', 'sadness', 'fear', 'happiness'], dtype=object)

In [30]:
df.discrete_emotion_collection_phase.unique()

array(['anger', 'neutral', 'sadness', 'happiness', 'fear'], dtype=object)

# Do all of the files exist?

In [3]:
def check_exists(i: int) -> bool:
    path = audiopath / Path(str(i) + ".wav")
    return path.exists()

#check_exists(df.utterance_id[0])
all(df.utterance_id.apply(check_exists))

True

Files exist.

# Converting the audio to 16kHz

In [34]:
%%bash
files=$(ls cres_v2.1/audio_files)
mkdir cres_v2.1/audio_files_16000
for file in $files
do
    ffmpeg -i "cres_v2.1/audio_files/$file" -ac 1 -ar 16000 "cres_v2.1/audio_files_16000/$file"
done

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/peterr/anaconda3 --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Guessed Channel Layout for Input Stream #0.0 : mono
Input #0, wav, from 'cres_v2.1/audio_files/1.wav':
  Duration: 00:00:06.70, bitrate: 176 kb/s
    Stream #

# Does it matter which target column we take?

In [35]:
df.columns

Index(['utterance_id', 'gender', 'age', 'speaker_id', 'expression',
       'manual_transcription', 'discrete_emotion_collection_phase',
       'discrete_emotion_annotation_phase', 'valence_annotation_phase',
       'arousal_annotation_phase', 'split'],
      dtype='object')

In [36]:
(df.discrete_emotion_annotation_phase == df.discrete_emotion_collection_phase).sum()

869

In [38]:
df.discrete_emotion_annotation_phase.isna().sum()

0

In [39]:
df.discrete_emotion_collection_phase.isna().sum()

0

According to the email from Branimir Dopuljić (2022-08-22T18:41) the target column should be the `_annotation_phase` responses.

Let's correct the paths, check that they all exist in the converted folder and save the dataset to a new file:


In [9]:
df["path"] = df.utterance_id.apply(
    lambda i: str(audiopath_16000 / Path(str(i) + ".wav"))
)
all([Path(i).exists() for i in df.path]
)



True

In [12]:
df["target"] = df['discrete_emotion_annotation_phase']

In [13]:
df.to_json("metadata.jsonl", orient="records", lines=True)